In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
import joblib
import pandas as pd



In [2]:
df= pd.read_csv('D:/bootcamp_experience/visualizacion/df_final.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24527 entries, 0 to 24526
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_recibo        24527 non-null  int64  
 1   producto_id      24527 non-null  int64  
 2   pedido_id        24527 non-null  int64  
 3   cantidad         24527 non-null  int64  
 4   valor_unitario   24527 non-null  int64  
 5   valor_total      24527 non-null  int64  
 6   ciudad           24527 non-null  object 
 7   costo_envio      24527 non-null  float64
 8   latitud          24527 non-null  float64
 9   longitud         24527 non-null  float64
 10  vendedor_id      24527 non-null  int64  
 11  fecha_compra     24527 non-null  object 
 12  anio             24527 non-null  int64  
 13  mes              24527 non-null  object 
 14  num_mes          24527 non-null  int64  
 15  producto         24527 non-null  object 
 16  tipo_producto    24527 non-null  object 
 17  marca       

In [4]:
df['fecha_compra'] = pd.to_datetime(df['fecha_compra'] )

In [5]:
df.head()

,id_recibo,producto_id,pedido_id,cantidad,valor_unitario,valor_total,ciudad,costo_envio,latitud,longitud,...,mes,num_mes,producto,tipo_producto,marca,sku,condicion,nombre_vendedor,ingreso_neto,estacion
0,1,41518,341,3,260,780,Bahia,156.0,-12.9704,-38.5124,...,January,1,Calca Jeans Basica,Calca,Banana Republic,255410.0,Nuevo con etiqueta,Unknown,624.0,Verano
1,2,4307,1174,5,6175,30875,Rio de Janeiro,6175.0,-22.9068,-43.1729,...,January,1,Bolsa Intrecciato Caramelo,Bolsa,Bottega Veneta,301305.0,Usado,Nadia Oliveira,24700.0,Verano
2,3,22407,1399,3,200,600,Paraíba,120.0,-7.2398,-36.7819,...,January,1,Cropped Listras Florzinhas,Cropped,Topshop,327426.0,Usado,Ana Duarte,480.0,Verano
3,4,3820,1652,6,139,834,Distrito Federal,166.8,-15.7939,-47.8828,...,January,1,Shorts Jeans Tye Dye,Shorts,Lezalez,252560.0,Usado,Millena Pereira,667.2,Verano
4,5,29012,2470,3,525,1575,Bahia,315.0,-12.9704,-38.5124,...,January,1,Casaco Branco Costuras,Casaco,Tigresse,257889.0,Usado,Daniel Siqueira,1260.0,Verano


In [6]:
df = df[df['anio']!=2021]

In [7]:
df.columns

Index(['id_recibo', 'producto_id', 'pedido_id', 'cantidad', 'valor_unitario',
       'valor_total', 'ciudad', 'costo_envio', 'latitud', 'longitud',
       'vendedor_id', 'fecha_compra', 'anio', 'mes', 'num_mes', 'producto',
       'tipo_producto', 'marca', 'sku', 'condicion', 'nombre_vendedor',
       'ingreso_neto', 'estacion'],
      dtype='object')

In [8]:
df=df.loc[:,['cantidad', 'valor_unitario', 'anio', 'fecha_compra','tipo_producto','marca', 'estacion','ingreso_neto']]

In [9]:
df.head()

,cantidad,valor_unitario,anio,fecha_compra,tipo_producto,marca,estacion,ingreso_neto
0,3,260,2019,2019-01-05,Calca,Banana Republic,Verano,624.0
1,5,6175,2019,2019-01-05,Bolsa,Bottega Veneta,Verano,24700.0
2,3,200,2019,2019-01-05,Cropped,Topshop,Verano,480.0
3,6,139,2019,2019-01-05,Shorts,Lezalez,Verano,667.2
4,3,525,2019,2019-01-05,Casaco,Tigresse,Verano,1260.0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22712 entries, 0 to 22711
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   cantidad        22712 non-null  int64         
 1   valor_unitario  22712 non-null  int64         
 2   anio            22712 non-null  int64         
 3   fecha_compra    22712 non-null  datetime64[ns]
 4   tipo_producto   22712 non-null  object        
 5   marca           22712 non-null  object        
 6   estacion        22712 non-null  object        
 7   ingreso_neto    22712 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 1.6+ MB


In [11]:
df = df.groupby(['anio','tipo_producto','estacion','fecha_compra'])['ingreso_neto'].sum().reset_index()
df.sample(5)

,anio,tipo_producto,estacion,fecha_compra,ingreso_neto
2067,2019,Clutch,Verano,2019-02-24,6400.0
798,2019,Bolsa,Invierno,2019-08-02,9880.0
1819,2019,Casaco,Invierno,2019-09-22,3360.0
6526,2020,Capa,Primavera,2020-12-20,1432.0
7277,2020,Cropped,Primavera,2020-10-30,1120.0


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam




# Convertir fechas a datetime
df['fecha_compra'] = pd.to_datetime(df['fecha_compra'])

# Codificación de variables categóricas
df['tipo_producto_encoded'] = df['tipo_producto'].astype('category').cat.codes
df['estacion_encoded'] = df['estacion'].astype('category').cat.codes

# Variables predictoras y variable objetivo
X = df[['anio', 'tipo_producto_encoded', 'estacion_encoded']]
y = df['ingreso_neto']

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos (es importante para las redes neuronales)
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

# Construcción del modelo de red neuronal
model_new = Sequential()
model_new.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model_new.add(Dense(32, activation='relu'))
model_new.add(Dense(1, activation='linear'))

# Compilación del modelo
model_new.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

# Entrenamiento del modelo
model_new.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=10, validation_split=0.1)

# Evaluación del modelo en el conjunto de prueba
loss = model_new.evaluate(X_test_scaled, y_test_scaled)
print(f"Error cuadrático medio en el conjunto de prueba: {loss}")

# Predicción con nuevos datos (ejemplo)
nuevos_datos = pd.DataFrame({
    'anio': [2021],
    'tipo_producto_encoded': [df['tipo_producto_encoded'][df['tipo_producto'] == 'Oculos'].values[0]],
    'estacion_encoded': [df['estacion_encoded'][df['estacion'] == 'Verano'].values[0]]
})

nuevos_datos_scaled = scaler_X.transform(nuevos_datos)
prediccion_scaled = model_new.predict(nuevos_datos_scaled)
prediccion = scaler_y.inverse_transform(prediccion_scaled)

print(f"Predicción de ingreso neto para los nuevos datos: {prediccion[0][0]}")


Epoch 1/100


d:\bootcamp_experience\visualizacion\env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


720/720 ━━━━━━━━━━━━━━━━━━━━ 2s 967us/step - loss: 0.0058 - val_loss: 0.0037
Epoch 2/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 1s 839us/step - loss: 0.0041 - val_loss: 0.0036
Epoch 3/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 1s 794us/step - loss: 0.0036 - val_loss: 0.0038
Epoch 4/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 1s 943us/step - loss: 0.0040 - val_loss: 0.0037
Epoch 5/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 1s 805us/step - loss: 0.0048 - val_loss: 0.0038
Epoch 6/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 1s 856us/step - loss: 0.0040 - val_loss: 0.0036
Epoch 7/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 1s 777us/step - loss: 0.0049 - val_loss: 0.0037
Epoch 8/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 1s 813us/step - loss: 0.0045 - val_loss: 0.0038
Epoch 9/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 1s 828us/step - loss: 0.0042 - val_loss: 0.0037
Epoch 10/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 1s 805us/step - loss: 0.0043 - val_loss: 0.0037
Epoch 11/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 1s 800us/step - loss: 0.0044 - val_loss: 0.0042
Epoch 12/100
720/720 ━━━━━━━━━━

In [14]:
from tensorflow import keras
# Guardar el modelo y los scalers
model_new.save('model_new.h5')
joblib.dump(scaler_X, 'scaler_X.pkl')
joblib.dump(scaler_y, 'scaler_y.pkl')

['scaler_y.pkl']